In [1]:
import pandas as pd

In [32]:
df = pd.read_csv("/data/lux70/data/pfam/organism_counts.csv")
df.head()

,organism_index,organism_id,counts
0,0,SYNS9,2126
1,1,SCHAZ,4805
2,2,9ENTE,108317
3,3,APHAT,14121
4,4,9GAMM,2601336


In [33]:
!head /data/lux70/data/uniref90/speclist_parse.txt 

AADNV V  648330: N=Aedes albopictus densovirus (isolate Boublik/1994)
AAV2  V   10804: N=Adeno-associated virus 2
AAV2S V  648242: N=Adeno-associated virus 2 (isolate Srivastava/1982)
ABABO E 3053407: N=Abaeis boisduvaliana
ABAMA E  118452: N=Abacion magnum
ABAME E 3053408: N=Abaeis mexicana
ABANI E   72259: N=Abaeis nicippe
ABAPA E  102642: N=Abax parallelepipedus
ABAST E  392897: N=Abalistes stellaris
ABBRI E   75332: N=Abbottina rivularis


In [34]:
df = pd.read_csv("/data/lux70/data/uniref90/speclist_parse.txt", sep=": N=",header=None)
df.columns = ['uniref_code','name']

In [35]:
df['uniref_identifier'] = df.uniref_code.str[:5]
df['uniref_code'] = df.uniref_code.str[7:].astype(int)

In [36]:
df.head()

,uniref_code,name,uniref_identifier
0,648330,Aedes albopictus densovirus (isolate Boublik/1...,AADNV
1,10804,Adeno-associated virus 2,AAV2
2,648242,Adeno-associated virus 2 (isolate Srivastava/1...,AAV2S
3,3053407,Abaeis boisduvaliana,ABABO
4,118452,Abacion magnum,ABAMA


In [7]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()
#ncbi.update_taxonomy_database()  # Download NCBI taxonomy

In [30]:
for taxon_id in df['uniref_code'][1:40:4]:
    print(f"{taxon_id} ======")
    lineage = ncbi.get_lineage(taxon_id)
    names = ncbi.get_taxid_translator(lineage)
    
    # Print higher-order taxonomy names
    print(len(lineage), [names[taxid] for taxid in lineage])

10804 ======
12 ['root', 'Viruses', 'Monodnaviria', 'Shotokuvirae', 'Cossaviricota', 'Quintoviricetes', 'Piccovirales', 'Parvoviridae', 'Parvovirinae', 'Dependoparvovirus', 'Dependoparvovirus primate1', 'adeno-associated virus 2']
3053408 ======
31 ['root', 'cellular organisms', 'Eukaryota', 'Opisthokonta', 'Metazoa', 'Eumetazoa', 'Bilateria', 'Protostomia', 'Ecdysozoa', 'Panarthropoda', 'Arthropoda', 'Mandibulata', 'Pancrustacea', 'Hexapoda', 'Insecta', 'Dicondylia', 'Pterygota', 'Neoptera', 'Endopterygota', 'Amphiesmenoptera', 'Lepidoptera', 'Glossata', 'Neolepidoptera', 'Heteroneura', 'Ditrysia', 'Obtectomera', 'Papilionoidea', 'Pieridae', 'Coliadinae', 'Abaeis', 'Abaeis mexicana']
75332 ======
30 ['root', 'cellular organisms', 'Eukaryota', 'Opisthokonta', 'Metazoa', 'Eumetazoa', 'Bilateria', 'Deuterostomia', 'Chordata', 'Craniata', 'Vertebrata', 'Gnathostomata', 'Teleostomi', 'Euteleostomi', 'Actinopterygii', 'Actinopteri', 'Neopterygii', 'Teleostei', 'Osteoglossocephalai', 'Clupeo

In [45]:
def parse_level(taxon_id, level):
    try:
        lineage = ncbi.get_lineage(taxon_id)
        names = ncbi.get_taxid_translator(lineage)
         
        ranks = ncbi.get_rank(lineage)
        
        # Find the kingdom in the lineage
        levels = [names[taxid] for taxid in lineage if ranks[taxid] == level]
        return levels[0]
        
    except:
        return ""

In [41]:
import warnings
warnings.filterwarnings("ignore")
df['kingdom'] = df.uniref_code.map(lambda x: parse_kingdom(x, "kingdom"))

In [44]:
import numpy as np

np.unique(df.kingdom.values, return_counts=True)

(array(['', 'Bamfordvirae', 'Fungi', 'Heunggongvirae', 'Loebvirae',
        'Metazoa', 'Orthornavirae', 'Pararnavirae', 'Sangervirae',
        'Shotokuvirae', 'Trapavirae', 'Viridiplantae', 'Zilligvirae'],
       dtype=object),
 array([ 5540,   176,  1102,   380,    14, 13270,  1744,   363,    10,
          234,     2,  4802,    14]))

In [ ]:
import warnings
warnings.filterwarnings("ignore")
df['superkingdom'] = df.uniref_code.map(lambda x: parse_level(x, "superkingdom"))